In [1]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
from langchain_openai import OpenAIEmbeddings

index_name = "recipes"

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

index = pc.Index(index_name)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

vectorstore = PineconeVectorStore.from_existing_index(
    index_name,
    embeddings,
)

docs = vectorstore.similarity_search("tofu")

docs

/Users/junghaein/Documents/FULLSTACK_GPT/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(id='aede20c7-ffcb-4103-b46d-ccbe3e913f19', metadata={'row': 297.0, 'source': './recipes.csv'}, page_content=': 297\nhref: https://veganuary.com/recipes/crispy-salt-pepper-spiced-tofu/\ntitle: Crispy Salt and Pepper Spiced Tofu\ningredients: Ingredients'),
 Document(id='ff2752d2-55ea-4cb3-979d-bbb608ca927e', metadata={'row': 1168.0, 'source': './recipes.csv'}, page_content='Place tofu on several layers of heavy-duty paper towels or dish cloth, let stand 20 minutes. Cut into 1/2-inch cubes.\nHeat oil in a medium saucepan over medium heat. Add tofu and dash of garlic powder, saute 3 minutes, add tamari and sesame seeds, cook another 2 minutes or so, until tofu is lightly golden on each side. Remove tofu from pan and set aside.\nIn same pan over medium heat, add quinoa and cook 3 minutes, stirring frequently. Add broth and salt, bring to boil. Cover, reduce heat to low and simmer 15 minutes. Remove from heat, uncover, add tofu but do not stir, and let set 10 min to soak up the re